In [13]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

### Load Data

In [55]:
demands = pd.read_excel(open('demands21.xlsx', 'rb'))
assuta_items = pd.read_excel("Assuta_items.xlsx")
refresh21 = pd.read_excel("refresh21.xlsx")
refresh22 = pd.read_excel("refresh22.xlsx")
initial_inventory = pd.read_excel("Initial_inventory.xls")

### Pre Processing

In [56]:
demands.fillna(0,inplace=True)

assuta_items = pd.DataFrame(assuta_items.iloc[:,0])

refresh21.rename(columns = {f'{refresh21.columns[0]}':'SKU_ID'}, inplace = True)
refresh22.rename(columns = {f'{refresh22.columns[0]}':'SKU_ID'}, inplace = True)
refresh = pd.merge(refresh21, refresh22, on ='SKU_ID', how ='inner')
refresh.fillna(0,inplace=True)

initial_inventory = initial_inventory.loc[:,['SKU','Total Qty']]
initial_inventory.rename(columns = {f'{initial_inventory.columns[0]}':'SKU_ID',f'{initial_inventory.columns[1]}':'Total Qty'}, inplace = True)


In [57]:
assuta_items

,SKU_ID
0,AS100016
1,AS120257
2,AS110041
3,AS180293
4,AS040107
...,...
1285,AS400185
1286,AS010122
1287,AS000201
1288,ASA00006


In [ ]:
# Create one dataframe with all the data and add a 'YEAR' column
for i in range(len(years)):
     demands[i]["YEAR"]=int(years[i])
total_demands = pd.concat(demands)

In [ ]:
# Load assuta items and remove them from the dataframe
assuta_items = pd.read_excel("Assuta_items.xlsx")
total_demands = total_demands[~total_demands["SKU_ID"].isin(assuta_items["SKU_ID"])].copy()

In [ ]:
# Group A - find the ~20% with highest demands
# for every product, calculate the average demand for the 4 years
import matplotlib.pyplot as plt
data_b = total_demands.copy()
annual_avg = ((data_b.groupby("SKU_ID")["TOTAL_FOR_YEAR"].mean())).to_frame()
annual_avg.columns = ['YEARLY_AVG']
annual_avg.reset_index(inplace=True)
# annual_avg.head()
data_c = total_demands.copy()
data_merged = pd.merge(data_c, annual_avg, how='inner', on = 'SKU_ID')
good_sku_ids = data_merged[["SKU_ID", "YEARLY_AVG"]].drop_duplicates().sort_values(by=['YEARLY_AVG'], ascending=False)
# a.reset_index(inplace=True)
# a.to_csv(r'good_sku_ids.csv', index = False)
good_sku_ids['CUMSUM'] = good_sku_ids['YEARLY_AVG'].cumsum()
good_sku_ids['CUMSUM_PERC'] = 100*good_sku_ids['CUMSUM']/good_sku_ids['YEARLY_AVG'].sum()
plt.plot(np.arange(len(good_sku_ids['CUMSUM_PERC'])),good_sku_ids['CUMSUM_PERC'])
good_sku_ids['Up_To'] = good_sku_ids['CUMSUM_PERC'] <= 70
good_sku_ids = good_sku_ids[good_sku_ids['Up_To']]
a_list_sku_ids = good_sku_ids['SKU_ID']
a_list_sku_ids

In [ ]:
total_demands = total_demands[total_demands["SKU_ID"].isin(list(a_list_sku_ids))].copy()

In [ ]:
# Fill Nulls with 0.1
total_demands.fillna(0.1,inplace=True)

In [ ]:
total_demands

In [ ]:
# Add quarters columns
total_demands["P1"] = total_demands["'01'"] + total_demands["'02'"]
total_demands["P2"] = total_demands["'03'"] + total_demands["'04'"]
total_demands["P3"] = total_demands["'05'"] + total_demands["'06'"]
total_demands["P4"] = total_demands["'07'"] + total_demands["'08'"]
total_demands["P5"] = total_demands["'09'"] + total_demands["'10'"]
total_demands["P6"] = total_demands["'11'"] + total_demands["'12'"]

# Add quarters columns
total_demands["Q1"] = total_demands["'01'"] + total_demands["'02'"] + total_demands["'03'"]
total_demands["Q2"] = total_demands["'04'"] + total_demands["'05'"] + total_demands["'06'"]
total_demands["Q3"] = total_demands["'07'"] + total_demands["'08'"] + total_demands["'09'"]
total_demands["Q4"] = total_demands["'10'"] + total_demands["'11'"] + total_demands["'12'"]

In [ ]:
total_demands

In [ ]:
# total_demands['demand_mean'] = np.mean(total_demands.iloc[:,1:13].to_numpy(),axis=1)
# total_demands['demand_stdv'] = np.std(total_demands.iloc[:,1:13].to_numpy(),axis=1)

In [ ]:
# Make 3 lists - less (r<9.5), good (9.5<r>10.5), more (r>10.5) while r is the number of refreshes per month
refresh = pd.read_csv('refresh.csv')
refresh = refresh.iloc[:-1,:] # Remove the last row ('sum' per month)
refresh.fillna(0,inplace=True)
refresh['mean'] = np.mean(refresh.iloc[:,1:].to_numpy(),axis=1)
sku_ids_0to95 = list(refresh[(refresh['mean']>=0)&(refresh['mean']<9.5)]['SKU_ID'])
sku_ids_95to105 = list(refresh[(refresh['mean']>=9.5)&(refresh['mean']<=10.5)]['SKU_ID'])
sku_ids_over105 = list(refresh[refresh['mean']>10.5]['SKU_ID'])

In [ ]:
hist = refresh['mean'].hist(bins=10,grid=False,figsize=[10,5],legend=True)

In [ ]:
lst1 = list(total_demands['SKU_ID'])
lst2 = sku_ids_0to95 + sku_ids_over105
relevant_sku_ids = list(set(lst1) & set(lst2)) # sku_ids of products that is not good (less or more)
print(f'The number of relevant sku_ids is: {len(relevant_sku_ids)}')

In [ ]:
# Stay with only the relevant products
total_demands = total_demands[total_demands['SKU_ID'].isin(relevant_sku_ids)]
total_demands

### MinMax

In [ ]:
data = total_demands.copy().iloc[:,[0,1,2,3,4,5,6,7,8,9,10,11,12,18]]
data.replace(0.1,0,inplace=True)
data

In [ ]:
# data[data.SKU_ID=='ULM00180']

In [ ]:
# Add quarters columns
data["Q1"] = (data["'01'"] + data["'02'"] + data["'03'"]) / 3
data["Q2"] = (data["'04'"] + data["'05'"] + data["'06'"]) / 3
data["Q3"] = (data["'07'"] + data["'08'"] + data["'09'"]) / 3
data["Q4"] = (data["'10'"] + data["'11'"] + data["'12'"]) / 3

In [ ]:
# data[data.SKU_ID=='ULM00180']

In [ ]:
data_Qs = data.iloc[:,data.columns.isin(['SKU_ID','Q1','Q2','Q3','Q4','YEAR'])]
data_Qs.reset_index(inplace=True,drop=True)
data_Qs

In [ ]:
# data_Qs[data_Qs.SKU_ID=='ULM00180']